In [4]:
import pandas as pd
import numpy as np

def get_removedevice(input_df: pd.DataFrame, device: str) -> pd.DataFrame:
    input_df['index'] = input_df.index 
    input_df = input_df.sort_values('millisSinceGpsEpoch')
    input_df.index = input_df['millisSinceGpsEpoch'].values # illisSinceGpsEpochをindexにする

    output_df = pd.DataFrame() 
    for _, subdf in input_df.groupby('collectionName'):

        phones = subdf['phoneName'].unique()

        # 1つのコレクションにphoneが1種類であるか、対象のデバイスがコレクションに含まれていない時
        if (len(phones) == 1) or (not device in phones):
            output_df = pd.concat([output_df, subdf])
            continue

        origin_df = subdf.copy()
        
        # 対象のデバイスの位置を削除
        _index = subdf['phoneName']==device
        subdf.loc[_index, 'latDeg'] = np.nan
        subdf.loc[_index, 'lngDeg'] = np.nan
        
        # Nanの周りに値が存在していれば、そのNanを補間
        # indexを基準として、線形的に補間していく
        subdf = subdf.interpolate(method='index', limit_area='inside')
        
        # 値が存在しないところは、元の値を使う
        _index = subdf['latDeg'].isnull()
        subdf.loc[_index, 'latDeg'] = origin_df.loc[_index, 'latDeg'].values
        subdf.loc[_index, 'lngDeg'] = origin_df.loc[_index, 'lngDeg'].values

        output_df = pd.concat([output_df, subdf])

    output_df.index = output_df['index'].values
    output_df = output_df.sort_index()

    del output_df['index']
    
    return output_df

submission = pd.read_csv('../output/sub_nb005.csv')
submission['collectionName'] = submission['phone'].map(lambda x: x.split('_')[0])
submission['phoneName'] = submission['phone'].map(lambda x: x.split('_')[1])
submission = get_removedevice(submission, 'SamsungS20Ultra')

submission = submission.drop(columns=['collectionName', 'phoneName'], axis=1)
submission.to_csv('../output/sub_nb016.csv.csv', index=False)